<a href="https://colab.research.google.com/github/venanen/machine-learning-experiments/blob/master/%D0%9C%D0%B0%D1%80%D0%BA_%D0%B3%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Импорты

In [ ]:
import pandas as pd

In [ ]:
!pip install transformers==4.19.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 12.6 MB/s eta 0:00:00


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import StoppingCriteria, StoppingCriteriaList
import torch

In [ ]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else None

In [ ]:
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Генерация

In [ ]:
path = 'AnyaSchen/news_gpt-3'
tokenizer = GPT2Tokenizer.from_pretrained(path)
model = GPT2LMHeadModel.from_pretrained(path).to(DEVICE)

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/983 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [ ]:
SPECIAL_TOKENS = {'bos_token':'<bos>','eos_token' :'<eos>', 'pad_token':'<pad>', 'sep_token': '<sep>'}
tokenizer.add_special_tokens(SPECIAL_TOKENS)

3

In [ ]:
class KeywordsStoppingCriteria(StoppingCriteria):
    def __init__(self, keywords_ids:list):
      self.keywords = keywords_ids

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
      if input_ids[0][-1] in self.keywords:
        print(input_ids)
        return True
      return False

In [ ]:
stop_criteria = KeywordsStoppingCriteria(tokenizer.encode(tokenizer.eos_token, return_tensors="pt").to(DEVICE))

## Задай свое начало предложения:



In [ ]:
inp = input('Введите начало заголовка или просто нажмите Enter:')

Введите начало заголовка или просто нажмите Enter:ученые


In [ ]:
inp = inp if len(inp) > 0 else tokenizer.bos_token #токен начала предложения
input_ids = tokenizer.encode(inp, return_tensors="pt").to(DEVICE)

In [ ]:
input_ids

tensor([[  610, 27630]], device='cuda:0')

In [ ]:
with torch.no_grad():
      out = model.generate(input_ids,
                          do_sample=True,
                          num_beams=3,
                          temperature=2.0,
                          top_p=0.9,
                          max_length = 200,
                          stopping_criteria=StoppingCriteriaList([stop_criteria]),
                          eos_token_id=tokenizer.eos_token_id,
                          bos_token_id=tokenizer.bos_token_id,
                          ).to(DEVICE)
print(tokenizer.batch_decode(out, skip_special_tokens=False)[0])

Setting `pad_token_id` to `eos_token_id`:50258 for open-end generation.


ученые выяснили, в каком возрасте девочки начинают считать себя глупее мальчиков <sep> По мнению американских ученых, девочек приучают считать себя глупее, еще начиная с младенчества. Они выяснили, как это происходит, и чем чревата такая тенденция для девочек. <eos>
